### Importing Libraries

In [2]:
import numpy as np
from astropy.table import Table, vstack
import lcdata
import sncosmo
import warnings

warnings.filterwarnings("ignore")


### Load the pretrained parsnip model with the wavelength bins

In [3]:
#Loading the wavelength grid from parsnip
waves = np.load('waves.npy')

#### Creating narrower bands for the spectra

In [24]:
"""Bands should be functions of wavelength (e.g: band no. n picks out nth 10 wavelengths)"""
bands = []
for i in range(0,300,10):
    wavelengths = waves[i:i+10]
    transmission = np.ones_like(wavelengths)
    band = sncosmo.Bandpass(wavelengths, transmission, name = f'band_{int(i/10)}')
    sncosmo.registry.register(band, force=True)
    bands.append(band)

## Generating lc + Spectra

In [4]:
#Define the function to generate light curves from, the default parameters output ~450-550 Supernovae.

def generate_lc(area = 4., tmin = 56075, tmax= 56425, zmax = 0.8, skynoise_min = 0.5, skynoise_max = 1,
                dust = sncosmo.CCM89Dust(), n_points = 300):
    
    spectra_t0 = []
    #Initialize the model
    model = sncosmo.Model(source='salt2-extended',
                      effects=[dust], 
                      effect_names=['host'], 
                      effect_frames=['rest']) 
    phases = np.random.uniform(tmin,tmax, size=n_points)
    #Observation parameters
    obs = Table({'time': phases,
             'band': ['lsstu', 'lsstg', 'lsstr', 'lssti', 'lsstz', 'lssty']*(int(n_points/6)),
             'gain': [1.] * n_points,
             'skynoise': np.random.uniform(skynoise_min, skynoise_max, size=n_points),
             'zp': [28.]*n_points, 
             'zpsys':['ab']*n_points})
    
    
    
    """Use one phase with 30 bands in it for every light curve"""
    
    """Generate a normal light curve (ugriz), 
    + a spectrum at around maximum light using generated bands (i.e: 5 through 29)"""

    """Initiate 1000 to 11000 array and zero out the waves under 2000 when simulating and zero weight (infinite uncertainty)"""
    
    """Generate all possible set of wavelengths and set zero wave zero flux where we have no data"""
    #Get a redshift distribution
    redshifts = list(sncosmo.zdist(0., zmax, time=(tmax-tmin), area=area))
    params = []
    s2n_max = 60 
    print(f' These distributions output {len(redshifts)} SNe')
    #Loop over redshifts to get amplitudes
    spectra_list = []
    for z in redshifts:
        mabs = np.random.normal(-19.3, 0.3)
        model.set(z=z)
        model.set_source_peakabsmag(mabs, 'lsstr', 'ab')
        x0 = model.get('x0')
        t0 = np.random.uniform(tmin, tmax)
        #Set Parameters
        p = {'z':z, 't0':t0, 'x0':x0, 'x1': np.random.normal(0., 1.), 'c': np.random.normal(0.,0.1)}
        params.append(p)
        model.set(**p)
        
        
        spectra = []
        used_bands = []
        c=0
        phase = np.random.uniform(t0-5, t0+5)
        for i in range(len(bands)):     
            try:
                spectra_flux = model.bandflux(bands[i], phase, zp=28., zpsys='ab')
                band = bands[i].name
                spectra.append(spectra_flux)
                used_bands.append(band)
                c=c+1
                #print(f'bands {used_bands} is used')
            except ValueError:
                pass
            
        spectra_table = Table({'time': [phase]*len(used_bands),
             'band': used_bands,
             'gain': [100] * len(spectra),
             'skynoise': np.random.uniform(skynoise_min, skynoise_max, size=len(spectra)),
             'zp': [28.]*len(spectra), 
             'zpsys':['ab']*len(spectra)})
        #print(spectra_table)
        #print(f'spectrum has {len(spectra)} elements')
        
        
        obs_spec = sncosmo.realize_lcs(spectra_table, model, [params[-1]])
        spec = lcdata.from_light_curves(obs_spec)
        spectra_list.append(spec)
    print(spec.light_curves)

    #Get light curve data and convert to dataset
    lcs = sncosmo.realize_lcs(obs, model, params)
    dataset = lcdata.from_light_curves(lcs)
    #Adding spectra into list of lists for each supernovae
    spectra = spectra_list
    return dataset, spectra
light_curve, spectra = generate_lc()


 These distributions output 539 SNe
[<Table length=16>
        time            flux     fluxerr     band     zp   zpsys
      float64         float32    float32    bytes7 float64  str2
 ------------------ ----------- ---------- ------- ------- -----
 56256.418758114734  -1.3625252 0.82990754 band_14    28.0    ab
 56256.418758114734  0.48245493  0.9534854 band_15    28.0    ab
 56256.418758114734 -0.54381675  0.5928373 band_16    28.0    ab
 56256.418758114734  0.72950375 0.74809074 band_17    28.0    ab
 56256.418758114734  0.22562928  0.7561792 band_18    28.0    ab
 56256.418758114734   1.1570276  0.7460244 band_19    28.0    ab
                ...         ...        ...     ...     ...   ...
 56256.418758114734   47.654285 0.93900037 band_23    28.0    ab
 56256.418758114734    85.96306  1.2474169 band_24    28.0    ab
 56256.418758114734   79.106346  1.1172229 band_25    28.0    ab
 56256.418758114734    90.48478   1.144234 band_26    28.0    ab
 56256.418758114734     82.5915  1.

In [7]:
for i in range(len(spectra)):
    spectra[i].light_curves[0].meta = dict(spectra[i].light_curves[0].meta)
    temp = light_curve.light_curves[i].meta 
    light_curve.light_curves[i].meta = dict(spectra[i].light_curves[0].meta)
    light_curve.light_curves[i] = vstack([light_curve.light_curves[i], spectra[i].light_curves[0]])
    light_curve.light_curves[i].meta = temp


In [13]:
light_curve.write_hdf5('./stacked_data_2.h5', 'w')